In [0]:
import os
import tensorflow as tf
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import cv2
from random import shuffle
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

from keras.applications.xception import *
from keras.preprocessing import image
from keras import backend as K
from keras.models import Model
from keras.models import Sequential

from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard

import matplotlib.pyplot as plt


Using TensorFlow backend.


In [0]:
# with subtract pixel mean:92%
  

In [0]:
num_classes=12
img_size=229
height=229
width=229
batch_size=16

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
dataset= "drive/My Drive/kaggle_competition/train/"
dir=os.listdir(dataset)
# test_dir= "D:/CNN/Datasets/test"

In [0]:
# print(os.getcwd())
# os.chdir('content/drive/kaggle_competition/train')

In [0]:
subtract_pixel_mean= False
data_augmentation= False

In [0]:
def create_train_data():
    total_train_images=0
    training_data= []
    label=0
    for i in dir:       
        class_dir= os.path.join(dataset, i) #return a list of string where string is the name of the image             
        images_list= os.listdir(class_dir)  
        print("class\t", i, len(images_list))  
        for image in tqdm(images_list):
            img_path= os.path.join(class_dir, image)            
            img_arr= cv2.imread(img_path, 1) # 0 for grayscale image >> img_arr is array of pixels
            new_img= cv2.resize(img_arr, (height, width))
            training_data.append([np.array(new_img),np.array(label)])
        print(label)
        label=label+1        
    shuffle(training_data)    
    return training_data

In [0]:
training_data = create_train_data()
training_data= np.array(training_data)
print(training_data.shape)

  1%|▏         | 9/654 [00:00<00:07, 85.26it/s]

class	 Loose Silky-bent 654


  5%|▍         | 25/516 [00:00<00:02, 241.47it/s]

0
class	 Scentless Mayweed 516


  3%|▎         | 10/385 [00:00<00:03, 98.36it/s]

...

9
class	 Black-grass 263


  1%|          | 4/390 [00:00<00:12, 31.50it/s]

10
class	 Charlock 390


100%|██████████| 390/390 [00:04<00:00, 95.62it/s] 

11
(4750, 2)


In [0]:
# def create_test_data():
#     testing_data=[]
    
#     images_list=os.listdir(test_dir) 
#     total_test_images= len(images_list) 
   
#     for image in tqdm(images_list):  
#         path= os.path.join(test_dir, image) 
#         new_image=cv2.resize(cv2.imread(path, 1), (height, width)) 
#         testing_data.append(np.array(new_image))
    
#     np.save('D:/CNN/Datasets/Plant_testing.npy', testing_data)
#     return testing_data

In [0]:
x_train= np.array([i[0] for i in training_data])
y_train= [i[1] for i in training_data]

#split train data into train and val set
x_train, x_val, y_train, y_val= train_test_split(x_train, y_train, test_size=0.2)

print("shape after splitting train, val and test sets\n")
print(x_train.shape)
print(len(y_train))
print(x_val.shape)
print(len(y_val))


shape after splitting train, val and test sets

(3800, 229, 229, 3)
3800
(950, 229, 229, 3)
950


In [0]:
#preprocessing

x_train = x_train.astype('float32') / 255
x_val = x_val.astype('float32') / 255
# x_test = x_test.astype('float32') / 255
    
#one hot encoding
y_train= to_categorical(y_train, num_classes) 
y_val= to_categorical(y_val, num_classes) 

print("after hot encoding", y_train.shape, y_val.shape)
    
#reshape data
x_train = x_train.reshape(x_train.shape[0], height, width, 3)
x_val = x_val.reshape(x_val.shape[0], height, width, 3)
# x_test = x_test.reshape(x_test.shape[0], height, width, 3)

print("shape after reshaping train, val and test sets\n")
print("train", x_train.shape)
print("val", x_val.shape)
# print("test", x_test.shape)

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_val -= x_train_mean
#     x_test -= x_train_mean

after hot encoding (3800, 12) (950, 12)
shape after reshaping train, val and test sets

train (3800, 229, 229, 3)
val (950, 229, 229, 3)


In [0]:
# def create_mask_for_plant(image):
#     image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

#     sensitivity = 35
#     lower_hsv = np.array([60 - sensitivity, 100, 50])
#     upper_hsv = np.array([60 + sensitivity, 255, 255])

#     mask = cv2.inRange(image_hsv, lower_hsv, upper_hsv)
#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11,11))
#     mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
#     return mask

# def segment_plant(image):
#     mask = create_mask_for_plant(image)
#     output = cv2.bitwise_and(image, image, mask = mask)
#     return output
  
# def sharpen_image(image):
#     image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
#     image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
#     return image_sharp


In [0]:
# image_segmented = segment_plant(image_bgr)
# image_sharpen = sharpen_image(image_segmented)
# cv2.imwrite(os.path.join('seg_test', file), image_sharpen)

In [0]:
# datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                              rotation_range=180,
#                              width_shift_range=0.3,
#                              height_shift_range=0.3,
#                              zoom_range=0.3,
#                              horizontal_flip=True,
#                              vertical_flip=True)

# datagen.fit(x_train)

In [0]:
base_model = Xception(weights='imagenet', input_shape=(img_size, img_size, 3), include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

W0707 09:53:57.915254 140587020388224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 09:53:57.938483 140587020388224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 09:53:57.943226 140587020388224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 09:53:57.966517 140587020388224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0707 09:53:57.968707 1405870203

In [0]:
model.compile(optimizer='Adadelta',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

W0707 09:54:04.540722 140587020388224 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [0]:

model.fit(x_train, y_train, batch_size= batch_size,
            
                    epochs=10, 
                    validation_data=(x_val, y_val), 
                    verbose=1)

model.save_weights('Xception.h5')

W0707 09:54:04.691733 140587020388224 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 3800 samples, validate on 950 samples
Epoch 1/10
3800/3800 [==============================] - 102s 27ms/step - loss: 1.1000 - acc: 0.6529 - val_loss: 1.4038 - val_acc: 0.7168
Epoch 2/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.4149 - acc: 0.8774 - val_loss: 0.5005 - val_acc: 0.8611
Epoch 3/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.2841 - acc: 0.9163 - val_loss: 0.4012 - val_acc: 0.9126
Epoch 4/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.1907 - acc: 0.9413 - val_loss: 0.4316 - val_acc: 0.8989
Epoch 5/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.1260 - acc: 0.9603 - val_loss: 0.3018 - val_acc: 0.9326
Epoch 6/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.1341 - acc: 0.9624 - val_loss: 0.4137 - val_acc: 0.9084
Epoch 7/10
3800/3800 [==============================] - 91s 24ms/step - loss: 0.0923 - acc: 0.9745 - val_loss: 0.2140 - val_acc: 0.9442
